In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Markdown as md

### With 100 points, three cases:

1. Ideal
2. Gaussian noise
3. Uniform noise

we have created a dataset with 100 points. The relation we want to investigate is

$$
2.5382 cos(x)+x^2-0.5
$$

In [ ]:
import numpy as np

X = 5 * np.random.randn(100,1)
a = 2 #weight for noise
y = 2.5382 * np.cos(X) + X ** 2 - 0.5 
n = y + a*np.random.randn(100,1)
u = y + a*np.random.rand(100,1)

In [ ]:
x = np.arange(np.min(X),np.max(X)+0.2, 0.2)
g = 2.5382 * np.cos(x) + x ** 2 - 0.5 
plt.plot(x,g, label='Ideal')
plt.scatter(X,n, label='Gaussian noise', s=5, color='r')
plt.scatter(X,u, label='Uniform noise', s=5, color='green')
plt.title('Graphic representation of dataset')
plt.xlabel('X')
plt.ylabel('y')
plt.legend()

from pysr import PySRRegressor

model = PySRRegressor(
    niterations=40,  # < Increase me for better results
    binary_operators=["+", "*"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "inv(x) = 1/x",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)    
    procs=8,
)

In [ ]:
from pysr import PySRRegressor

model = PySRRegressor(
    niterations=40,  # < Increase me for better results
    binary_operators=["+", "*"],
    unary_operators=[
        "cos",
        "sin",
        "exp",
        "log",
    ],
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)    
    warm_start=True,
    turbo=True,
    #batching=True, not so useful for this example
)

## 1. Ideal


In [ ]:
model.fit(X, y)

In [ ]:
p_1 = model.predict(X)
l_1 = model.latex()
p=[]
l=[]
p.append(p_1), l.append(l_1)

In [ ]:
plt.scatter(y, p_1,s=15)
plt.plot(y,y, color='r', lw=1)
plt.xlabel('Truth')
plt.ylabel('Prediction')
plt.show()

In [ ]:
md(f"The model has this equation as output: ${l[0]}$")

## 2. Gaussian noise


In [ ]:
model.fit(X,n)

In [ ]:
p_2 = model.predict(X)
l_2 = model.latex()
p.append(p_2), l.append(l_2)

In [ ]:
plt.scatter(y, p_2,s=15, label='prediction')
plt.plot(y,y, color='r', lw=1, label='true')
plt.legend()
plt.show()

In [ ]:
md(f"The model has this equation as output: ${l[1]}$")

## 3. Uniform noise

In [ ]:
model.fit(X,u)

In [ ]:
p_3 = model.predict(X)
l_3 = model.latex()
p.append(p_3), l.append(l_3)

In [ ]:
plt.scatter(y, p_3,s=15, label='prediction')
plt.plot(y,y, color='r', lw=1, label='true')
plt.legend()
plt.show()

In [ ]:
md(f"The model has this equation as output: ${l[2]}$")

## Comparison beetween ideal and noised

Gaussian

I compute the diferences between ideal and noised data, and later I used the discovered formulas to do the same:

In [ ]:
d_1 = y-n 
d_2 = p_1-p_2

In [ ]:
plt.plot(d_1, label='from data')
plt.plot(d_2, label='from model')
plt.legend()

seems that the model reduces the discrepances between ideal data and gaussian-noised data: I think that's fine because we find in both cases two formulas very similar. ($ x^2+a\cdot cos(x)+ b$, where  $a, b \in \mathbb{R}$)


Uniform

In [ ]:
r_1 = y-u
r_2 = p_1-p_3

In [ ]:
plt.plot(r_1, label='from data')
plt.plot(r_2, label='from model')
plt.legend()

## Tryhard #1: using only 50 points

In [ ]:
X = 5 * np.random.randn(50,1)
y = 2.5382 * np.cos(X) + X ** 2 - 0.5 
n = y + a*np.random.randn(50,1)
u = y + a*np.random.rand(50,1)

In [ ]:
x = np.arange(np.min(X),np.max(X)+0.2, 0.2)
g = 2.5382 * np.cos(x) + x ** 2 - 0.5 
plt.plot(x,g, label='ideal')
plt.scatter(X,u, label='uniform noise', color='r', s=8)
plt.scatter(X,n, label='gaussian noise', color='green', s=8, alpha=0.5)
plt.legend()

Ideal

In [ ]:
model.fit(X, y)

In [ ]:
l_4 = model.latex()
p_4 = model.predict(X)
p.append(p_4), l.append(l_4)

In [ ]:
md(f"The model has this equation as output: ${l[3]}$")

Gaussian

In [ ]:
model.fit(X, n)

In [ ]:
l_4 = model.latex()
p_4 = model.predict(X)
p.append(p_4), l.append(l_4)

In [ ]:
md(f"The model has this equation as output: ${l[4]}$")

uniform

In [ ]:
model.fit(X, u)

In [ ]:
l_4 = model.latex()
p_4 = model.predict(X)
p.append(p_4), l.append(l_4)
md(f"The model has this equation as output: ${l[5]}$")

It seems that the gaussian noise has a greater disturbance on the data, in fact the expression that I found is slightly different from the ideal one. The model suffers a bit with gaussian noise, while the model is reliable with the uniform noise

## Tryhard: 5(or 2) variables



In [ ]:
X = 5 * np.random.randn(100, 2)
a = 2 #weight for noise
y = 2.5382 * np.cos(X[:,1]) + X[:,0] ** 2 - 0.5
n = y + a*np.random.randn(100)
u = y + a*np.random.rand(100)

we have created a dataset with 100 points with 2 features each. The relation we want to investigate is

$$
2.5382 cos(x_1)+x_0^2-0.5
$$

## 1. Ideal

In [ ]:
model.fit(X,y)

In [ ]:
l_5 = model.latex()
p_5 = model.predict(X)
p.append(p_5), l.append(l_5)
md(f"The model has this equation as output: ${l[6]}$")

In [ ]:
plt.scatter(y, p[6],s=15)
plt.plot(y,y, color='r', lw=1)
plt.xlabel('Truth')
plt.ylabel('Prediction')
plt.show()

## 2. Gaussian

In [ ]:
model.fit(X,n)

In [ ]:
l_5 = model.latex()
p_5 = model.predict(X)
p.append(p_5), l.append(l_5)
md(f"The model has this equation as output: ${l[7]}$")

## 3. Uniform

In [ ]:
model.fit(X,u)

In [ ]:
l_5 = model.latex()
p_5 = model.predict(X)
p.append(p_5), l.append(l_5)
md(f"The model has this equation as output: ${l[8]}$")

## 5 (3 fake) variables

In [ ]:
X = 5 * np.random.randn(100, 5)
a = 2 #weight for noise
y = 2.5382 * np.cos(X[:,3]) + X[:,0] ** 2 - 0.5
n = y + a*np.random.randn(100)
u = y + a*np.random.rand(100)

we have created a dataset with 100 points with 5 features each. The relation we want to investigate is

$$
2.5382 cos(x_3)+x_0^2-0.5
$$

### Ideal

In [ ]:
model.fit(X,y)

In [ ]:
l_5 = model.latex()
p_5 = model.predict(X)
p.append(p_5), l.append(l_5)
md(f"The model has this equation as output: ${l[9]}$")

### Gaussian noise


In [ ]:
model.fit(X,n)

In [ ]:
l_5 = model.latex()
p_5 = model.predict(X)
p.append(p_5), l.append(l_5)
md(f"The model has this equation as output: ${l[10]}$")

### Uniform noise

In [ ]:
model.fit(X,u)

In [ ]:
l_5 = model.latex()
p_5 = model.predict(X)
p.append(p_5), l.append(l_5)
md(f"The model has this equation as output: ${l[11]}$")

It seems that the model fails with gaussian noise, both with one variable and with two variables. The model is anyway solid with uniform noise. 

Now we can try to increase the number of data, to see if the model can recover the expression that we want.

### fix #1: increasing the points

In [ ]:
X = 5 * np.random.randn(1000, 5)
a = 2 #weight for noise
y = 2.5382 * np.cos(X[:,3]) + X[:,0] ** 2 - 0.5
n = y + a*np.random.randn(1000)
model.fit(X,n)

In [ ]:
md(f"The model has this equation as output: ${model.latex()}$")

It seems that the problem persist: perhaps the problem is in the factor of amplification of the noise.

### fix #2: decreasing the noise

In [ ]:
X = 5 * np.random.randn(100, 5)
a = 0.5 #weight for noise
y = 2.5382 * np.cos(X[:,3]) + X[:,0] ** 2 - 0.5
n = y + a*np.random.randn(100)
model.fit(X,n)

In [ ]:
md(f"The model has this equation as output: ${model.latex()}$")

We finally recovered an expression that looks like the ideal one: we can compute the difference between these two quantities.

y is the dataset generated, and f(x) is the prediction from the model with gaussian noise

In [ ]:
plt.plot(y-model.predict(X))
plt.ylabel(r'$y-f(x)$')

Let's evaluate the percentage error:

In [ ]:
pc_err = 100*np.abs((y-model.predict(X))/y)
err_mean = np.mean(pc_err)
plt.plot(pc_err)
plt.ylabel('percentage error')

there are few critical points, probably these are the ones near zero, where y is small and so the noise is of the same order or pheraps of an higher order of magnitude of y.

In [ ]:
md(f'The mean value of the percentage error is: {err_mean}')

# Nice example: tying to interpolate the Debye model

The Debye model for specific heat has this integral form:

$$ c_v(T) = 9R\bigg(\frac{T}{T_D}\bigg)^3 \int_{0}^{\frac{T_D}{T}}\frac{x^4e^x}{(e^x-1)^2}dx$$

where

$$ x = \frac{hv_sn}{2Lk_bT} \quad T_D=\frac{hv_s}{2K_b}\sqrt[3]{\frac{6}{\pi}\frac{N}{V}}$$

if we consider a cube made by iron (L=$1\ m$), we know that $T_D=464\ K$ and $n=2$ because iron is a BCC solid.

In [ ]:
import numpy as np
from scipy.integrate import quad

T_D = 464
def integrand(x):
    return x**4 *np.exp(x)/ (np.exp(x) - 1)**2

T = np.arange(1, 2000, 0.5)  

def calculate_debye_integral(T):
    result, _ = quad(integrand, 0, T_D/T)
    return result

results = np.vectorize(calculate_debye_integral)(T)
C_v = results*9*8.314*(T/T_D)**3

plt.plot(T,C_v)
plt.xlabel(r"$T\ [K]$")
plt.ylabel(r"$c_v(T)\ [\frac{J}{K}]$")
plt.title("Debye model")
plt.grid(True)
plt.show()


In [ ]:
C_v1 = C_v.reshape(-1, 1)
T1 = T.reshape(-1, 1)

In [ ]:
debyemodel = PySRRegressor(
    niterations=50,  # < Increase me for better results
    binary_operators=["+", "*",'-','/',"^"],
    constraints={'^': (1, 1)},
    nested_constraints={"^": {"^": 2}},
    unary_operators=[
        "exp",
        "log",
        'sinh',
        'cosh',
        'erf',
    ],
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
    turbo=True,  
    cluster_manager="lsf",
    multithreading=True,
)


In [ ]:
debyemodel.fit(T1, C_v1)

In [ ]:
md(f"The model has this equation as output: ${debyemodel.latex()}$")

In [ ]:
from scipy.special import erf
#c_v_int = np.exp(np.exp(np.sinh(erf(0.695 * np.log(0.0356 * T))))) - 0.808
#plt.plot(T,c_v_int, label='Simbolic regression from PySR')
c_v_pysr = debyemodel.predict(T1)
plt.plot(T,c_v_pysr,linestyle='--',label='Simbolic regression from PySR', color='r')
plt.plot(T,C_v, label='Debye model')
plt.xlabel(r"$T\ [K]$")
plt.ylabel(r"$c_v(T)\ [\frac{J}{K}]$")
plt.title("Debye VS PySR")
plt.grid(True)
plt.legend()

In [ ]:
plt.plot(T,C_v-c_v_pysr)
plt.title('Differences between the two formulas')
plt.xlabel(r"$T\ [K]$")
plt.ylabel(r"$\Delta c_v(T)\ [\frac{J}{K}]$")
plt.grid(True)


That was a bold try :-)